In [1]:
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import pandas as pd
import json
import csv
import sys
import time

ckey = 'bXRPnBdpsgRdrkfbrHBHFInjl'
csecret = 'xm93rCpGvtV6E2lwx6392l5uUfYpz9dgCe7zREGtCWeWd5b6WB'
atoken = '1191115298788777987-qUAhRZ5dAw2wN1QewZxjROvtnVLHBs'
asecret = '595HhAG8XXlbN7BPYfLVUio1ozLqmLMnFDDIaHfzPmeSv'

def toDataFrame(tweets):

    DataSet = pd.DataFrame()

    DataSet['tweetID'] = [tweet.id for tweet in tweets]
    DataSet['tweetText'] = [tweet.text for tweet in tweets]
    DataSet['tweetRetweetCt'] = [tweet.retweet_count for tweet in tweets]
    DataSet['tweetFavoriteCt'] = [tweet.favorite_count for tweet in tweets]
    DataSet['tweetSource'] = [tweet.source for tweet in tweets]
    DataSet['tweetCreated'] = [tweet.created_at for tweet in tweets]
    DataSet['userID'] = [tweet.user.id for tweet in tweets]
    DataSet['userScreen'] = [tweet.user.screen_name for tweet in tweets]
    DataSet['userName'] = [tweet.user.name for tweet in tweets]
    DataSet['userCreateDt'] = [tweet.user.created_at for tweet in tweets]
    DataSet['userDesc'] = [tweet.user.description for tweet in tweets]
    DataSet['userFollowerCt'] = [tweet.user.followers_count for tweet in tweets]
    DataSet['userFriendsCt'] = [tweet.user.friends_count for tweet in tweets]
    DataSet['userLocation'] = [tweet.user.location for tweet in tweets]
    DataSet['userTimezone'] = [tweet.user.time_zone for tweet in tweets]
    DataSet['Coordinates'] = [tweet.coordinates for tweet in tweets]
    DataSet['GeoEnabled'] = [tweet.user.geo_enabled for tweet in tweets]
    DataSet['Language'] = [tweet.user.lang for tweet in tweets]
    tweets_place= []
    #users_retweeted = []
    for tweet in tweets:
        if tweet.place:
            tweets_place.append(tweet.place.full_name)
        else:
            tweets_place.append('null')
    DataSet['TweetPlace'] = [i for i in tweets_place]
    #DataSet['UserWhoRetweeted'] = [i for i in users_retweeted]

    return DataSet

OAUTH_KEYS = {'consumer_key':ckey, 'consumer_secret':csecret,'access_token_key':atoken, 'access_token_secret':asecret}
auth = tweepy.OAuthHandler(OAUTH_KEYS['consumer_key'], OAUTH_KEYS['consumer_secret'])
#auth = tweepy.AppAuthHandler('key', 'key')

api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)
else:
# I am trying to download from Dec 1st to Dec 7th but I am not able to

    cursor = tweepy.Cursor(api.search, q='#biden',lang='en')
    results=[]
    for item in cursor.items(10000):
        results.append(item)

    DataSet = toDataFrame(results)

Rate limit reached. Sleeping for: 700


In [2]:
DataSet["text"] = DataSet["tweetText"]
DataSet.shape

(10000, 20)

In [3]:
import re
import preprocessor as p

DataSet["text"] = DataSet["tweetText"]

pd.options.display.max_colwidth = 500

def preprocess_tweet(row):
    text = row['text']
    text = p.clean(text)
    return text

DataSet["text"]  = DataSet.apply(preprocess_tweet, axis=1)

DataSet["text"] = DataSet["text"].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')


DataSet[["tweetText", "text"]].head()

,tweetText,text
0,"The fact that #TrumpHasCovid is revealing that #DemocratsAreCorrupt , celebrating his diagnosis and wishing for his… https://t.co/TfmqxSgzGO",the fact that is revealing that celebrating his diagnosis and wishing for his
1,"RT @pollsofpolitics: Is @realDonaldTrump Responsible for the #coronavirus being in #America??\n\nPlease vote and retweet to spread poll, Than…",is responsible for the being in please vote and retweet to spread poll than
2,You are on target with your profound statement about both #Trumps condition and the #election . His decision to avo… https://t.co/xF1SYuWf99,you are on target with your profound statement about both condition and the his decision to avo
3,"RT @HelloMittyKitty: @KathiInCali @spokanetom @gtconway3d The email the Trump campaign attack dogs sent out was titled ""Lyin' Obama"" 👇 &amp; it…",the email the trump campaign attack dogs sent out was titled lyin obama amp it
4,RT @TruthPrinceof: @JoeBiden Those who live in glass houses shouldn’t throw stones. Let’s talk about your son’s dealings with China or Buri…,those who live in glass houses shouldnt throw stones lets talk about your sons dealings with china or buri


In [4]:
from nltk.tokenize import word_tokenize



def custom_tokenize(text):
    if not text:
        print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
    return word_tokenize(text)
DataSet['text2'] = DataSet['text'].apply(custom_tokenize)


DataSet[["text", "text2"]].head()


The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None

,text,text2
0,the fact that is revealing that celebrating his diagnosis and wishing for his,"[the, fact, that, is, revealing, that, celebrating, his, diagnosis, and, wishing, for, his]"
1,is responsible for the being in please vote and retweet to spread poll than,"[is, responsible, for, the, being, in, please, vote, and, retweet, to, spread, poll, than]"
2,you are on target with your profound statement about both condition and the his decision to avo,"[you, are, on, target, with, your, profound, statement, about, both, condition, and, the, his, decision, to, avo]"
3,the email the trump campaign attack dogs sent out was titled lyin obama amp it,"[the, email, the, trump, campaign, attack, dogs, sent, out, was, titled, lyin, obama, amp, it]"
4,those who live in glass houses shouldnt throw stones lets talk about your sons dealings with china or buri,"[those, who, live, in, glass, houses, shouldnt, throw, stones, lets, talk, about, your, sons, dealings, with, china, or, buri]"


In [5]:
from nltk.stem.snowball import SnowballStemmer

# Use English stemmer.
stemmer = SnowballStemmer("english")

DataSet['text3'] = DataSet['text2'].apply(lambda x: [stemmer.stem(y) for y in x]) 


DataSet[["tweetText", "text2", "text3"]].head()

,tweetText,text2,text3
0,"The fact that #TrumpHasCovid is revealing that #DemocratsAreCorrupt , celebrating his diagnosis and wishing for his… https://t.co/TfmqxSgzGO","[the, fact, that, is, revealing, that, celebrating, his, diagnosis, and, wishing, for, his]","[the, fact, that, is, reveal, that, celebr, his, diagnosi, and, wish, for, his]"
1,"RT @pollsofpolitics: Is @realDonaldTrump Responsible for the #coronavirus being in #America??\n\nPlease vote and retweet to spread poll, Than…","[is, responsible, for, the, being, in, please, vote, and, retweet, to, spread, poll, than]","[is, respons, for, the, be, in, pleas, vote, and, retweet, to, spread, poll, than]"
2,You are on target with your profound statement about both #Trumps condition and the #election . His decision to avo… https://t.co/xF1SYuWf99,"[you, are, on, target, with, your, profound, statement, about, both, condition, and, the, his, decision, to, avo]","[you, are, on, target, with, your, profound, statement, about, both, condit, and, the, his, decis, to, avo]"
3,"RT @HelloMittyKitty: @KathiInCali @spokanetom @gtconway3d The email the Trump campaign attack dogs sent out was titled ""Lyin' Obama"" 👇 &amp; it…","[the, email, the, trump, campaign, attack, dogs, sent, out, was, titled, lyin, obama, amp, it]","[the, email, the, trump, campaign, attack, dog, sent, out, was, titl, lyin, obama, amp, it]"
4,RT @TruthPrinceof: @JoeBiden Those who live in glass houses shouldn’t throw stones. Let’s talk about your son’s dealings with China or Buri…,"[those, who, live, in, glass, houses, shouldnt, throw, stones, lets, talk, about, your, sons, dealings, with, china, or, buri]","[those, who, live, in, glass, hous, shouldnt, throw, stone, let, talk, about, your, son, deal, with, china, or, buri]"


In [6]:
import datetime
year = datetime.date.today().year
month = datetime.date.today().month
month = '{:02d}'.format(month)
day = datetime.date.today().day
day = '{:02d}'.format(day)

today = datetime.date.today().strftime("%Y.%m.%d")
DataSet["keresoszo"] = "#biden"
DataSet["letoltes_datum"] = today

In [7]:
DataSet.to_csv('data/biden' + str(year) + str(month) + str(day) + '.csv',index=False)

In [8]:
DataSet.head()

,tweetID,tweetText,tweetRetweetCt,tweetFavoriteCt,tweetSource,tweetCreated,userID,userScreen,userName,userCreateDt,...,userTimezone,Coordinates,GeoEnabled,Language,TweetPlace,text,text2,text3,keresoszo,letoltes_datum
0,1312392237234434048,"The fact that #TrumpHasCovid is revealing that #DemocratsAreCorrupt , celebrating his diagnosis and wishing for his… https://t.co/TfmqxSgzGO",0,0,Twitter for Android,2020-10-03 14:01:07,1091926912576950272,Kevin_enjoys669,KEVIN 🎃 🎸 🤘🔞,2019-02-03 05:10:37,...,None,None,False,None,null,the fact that is revealing that celebrating his diagnosis and wishing for his,"[the, fact, that, is, revealing, that, celebrating, his, diagnosis, and, wishing, for, his]","[the, fact, that, is, reveal, that, celebr, his, diagnosi, and, wish, for, his]",#biden,2020.10.03
1,1312392223858671617,"RT @pollsofpolitics: Is @realDonaldTrump Responsible for the #coronavirus being in #America??\n\nPlease vote and retweet to spread poll, Than…",41,0,Twitter for iPhone,2020-10-03 14:01:03,3231411456,BrieRudd,@TrumpSharkCA,2015-05-31 08:19:48,...,None,None,True,None,null,is responsible for the being in please vote and retweet to spread poll than,"[is, responsible, for, the, being, in, please, vote, and, retweet, to, spread, poll, than]","[is, respons, for, the, be, in, pleas, vote, and, retweet, to, spread, poll, than]",#biden,2020.10.03
2,1312392204976095232,You are on target with your profound statement about both #Trumps condition and the #election . His decision to avo… https://t.co/xF1SYuWf99,0,0,Twitter for iPhone,2020-10-03 14:00:59,31045384,CULTURALPATHS,JACK HENNESSEY,2009-04-14 03:23:47,...,None,None,True,None,"South Carolina, USA",you are on target with your profound statement about both condition and the his decision to avo,"[you, are, on, target, with, your, profound, statement, about, both, condition, and, the, his, decision, to, avo]","[you, are, on, target, with, your, profound, statement, about, both, condit, and, the, his, decis, to, avo]",#biden,2020.10.03
3,1312392177301897216,"RT @HelloMittyKitty: @KathiInCali @spokanetom @gtconway3d The email the Trump campaign attack dogs sent out was titled ""Lyin' Obama"" 👇 &amp; it…",10,0,Twitter for Android,2020-10-03 14:00:52,927268721348952064,Winter611Love,"We are Ruthless, Vote Accordingly",2017-11-05 20:17:45,...,None,None,False,None,null,the email the trump campaign attack dogs sent out was titled lyin obama amp it,"[the, email, the, trump, campaign, attack, dogs, sent, out, was, titled, lyin, obama, amp, it]","[the, email, the, trump, campaign, attack, dog, sent, out, was, titl, lyin, obama, amp, it]",#biden,2020.10.03
4,1312392145517473793,RT @TruthPrinceof: @JoeBiden Those who live in glass houses shouldn’t throw stones. Let’s talk about your son’s dealings with China or Buri…,1,0,,2020-10-03 14:00:45,1282822496991850501,70MilesPerHour,MO,2020-07-13 23:41:47,...,None,None,False,None,null,those who live in glass houses shouldnt throw stones lets talk about your sons dealings with china or buri,"[those, who, live, in, glass, houses, shouldnt, throw, stones, lets, talk, about, your, sons, dealings, with, china, or, buri]","[those, who, live, in, glass, hous, shouldnt, throw, stone, let, talk, about, your, son, deal, with, china, or, buri]",#biden,2020.10.03
